In [11]:
from pathlib import Path
import pandas as pd

repo_root = Path.cwd()
for up in [repo_root, repo_root.parent, repo_root.parent.parent]:
    if (up / "src").exists():
        repo_root = up
        break

DATA = repo_root / "data" / "processed" / "flights_nativecadence_enu_kinematics.parquet"
df = pd.read_parquet(DATA, engine="fastparquet")

df

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,...,dE,dN,dU,vE,vN,vU,speed,heading_rad,heading_unwrapped,turn_rate
0,2017-06-12 00:00:10,001915,41.344620,-82.016418,240.196322,276.394398,0.32512,N645PM,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-06-12 00:00:20,001915,41.344620,-82.016418,240.196322,276.394398,0.32512,N645PM,False,False,...,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000,NaN,NaN
2,2017-06-12 00:00:30,001915,41.344620,-82.016418,240.196322,276.394398,0.32512,N645PM,False,False,...,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000,NaN,NaN
3,2017-06-12 00:00:40,001915,41.344620,-82.016418,240.196322,276.394398,0.32512,N645PM,False,False,...,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000,NaN,NaN
4,2017-06-12 00:00:50,001915,41.344620,-82.016418,240.196322,276.394398,0.32512,N645PM,False,False,...,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038550,2017-06-12 00:26:40,fa0a56,35.879196,-80.207806,128.016989,208.830968,-8.12800,,False,False,...,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000,NaN,NaN
1038551,2017-06-12 00:26:50,fa0a56,35.879196,-80.207806,128.016989,208.830968,-8.12800,,False,False,...,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000,NaN,NaN
1038552,2017-06-12 00:27:00,fa0a56,35.879196,-80.207806,128.016989,208.830968,-8.12800,,False,False,...,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000,NaN,NaN
1038553,2017-06-12 00:27:10,fa0a56,35.879196,-80.207806,128.016989,208.830968,-8.12800,,False,False,...,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000,NaN,NaN


In [39]:
# --- 10-run EKF eval with relaxed cadence + per-run tolerance ---

import numpy as np
import pandas as pd
from IPython.display import display
from src.ekf_model import CoordinatedTurnEKF, EKFParams

HORIZONS      = [60, 120, 300]
DT_MED_LO, DT_MED_HI = 10, 150         # relaxed cadence band (you lowered LO to 10)
MIN_LEN       = 30
MIN_DURATION  = max(HORIZONS) + 300    # >= 5 min beyond longest horizon
N_RUNS        = 10
SELECT_MODE   = "largest"              # or "random"

# Ensure numeric types (required + optional)
req_cols = ["E","N","U","dt"]
opt_cols = ["vE","vN","vU","turn_rate"]
for c in req_cols + opt_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
df["valid_req"] = df[req_cols].apply(np.isfinite).all(axis=1)
df["speed"] = np.sqrt(df["vE"]**2 + df["vN"]**2) if {"vE","vN"}.issubset(df.columns) else np.nan

# Per-run health (no .apply deprecation)
gb = df.groupby(["flight_id","run_id"], sort=False)
health = gb.agg(
    len=("time","size"),
    finite_frac=("valid_req","mean"),
    dt_med=("dt","median"),
    dt_max=("dt","max"),
    t_min=("time","min"),
    t_max=("time","max"),
    E_abs_max=("E", lambda s: s.abs().max()),
    N_abs_max=("N", lambda s: s.abs().max()),
    v_abs_p95=("speed", lambda s: np.nanpercentile(s.to_numpy(float), 95) if s.notna().any() else np.nan),
    omega_abs_p95=("turn_rate", lambda s: np.nanpercentile(np.abs(s.to_numpy(float)), 95) if s.notna().any() else np.nan),
).reset_index()
health["duration_s"] = (health["t_max"] - health["t_min"]).dt.total_seconds()

# Relaxed gates
HEALTHY = (
    (health["finite_frac"] > 0.95) &
    (health[["E_abs_max","N_abs_max"]].max(axis=1) < 2e6) &      # |E| or |N| < 2000 km
    (health["dt_med"].between(DT_MED_LO, DT_MED_HI, inclusive="both")) &
    (health["duration_s"] >= MIN_DURATION)
)
if not health["v_abs_p95"].isna().all():
    HEALTHY &= (health["v_abs_p95"] < 320)                       # ~620 kt (p95)
if not health["omega_abs_p95"].isna().all():
    HEALTHY &= (health["omega_abs_p95"] < 0.5)                    # ~29 deg/s (p95)

healthy = health[HEALTHY].copy()
if healthy.empty:
    raise RuntimeError("Still no runs passing health gates. Widen dt band or lower finite_frac to 0.9.")

# Pick 10
if SELECT_MODE == "largest":
    selected = healthy.sort_values("len", ascending=False).head(N_RUNS)
else:
    selected = healthy.sample(n=min(N_RUNS, len(healthy)), random_state=0)

print("Selected runs:")
display(selected[["flight_id","run_id","len","finite_frac","dt_med","duration_s"]])

# EKF with per-run dynamic tolerance
ekf = CoordinatedTurnEKF(EKFParams(), use_wind=False)

def dyn_tol(dt_med: float) -> int:
    # tolerance = clamp(3*dt_med, 15, 90) seconds
    return int(max(15, min(90, 3.0*float(dt_med))))

per_run_rows, sse2d, cnt = [], {h:0.0 for h in HORIZONS}, {h:0 for h in HORIZONS}

for _, row in selected.iterrows():
    fid, rid = int(row["flight_id"]), int(row["run_id"])
    tol_s = dyn_tol(row["dt_med"])
    track = (
        df[(df.flight_id==fid) & (df.run_id==rid)]
        [["time","E","N","U","vE","vN","vU","turn_rate","dt"]]
        .sort_values("time").reset_index(drop=True)
    )
    if len(track) < MIN_LEN:
        continue
    filt = ekf.filter_track(track)
    res  = ekf.horizon_errors(track, filt, horizons_s=HORIZONS, tol_s=tol_s)

    for h, d in res.items():
        if d is None or len(d)==0:
            continue
        d = d.copy()
        d["err2d_m"] = np.sqrt(d["eE"]**2 + d["eN"]**2)
        per_run_rows.append({
            "flight_id": fid, "run_id": rid, "H_s": int(h),
            "pairs": int(len(d)),
            "RMSE2D_m": float(np.sqrt(np.mean(d["err2d_m"]**2))),
            "RMSE3D_m": float(np.sqrt(np.mean(d["err_m"]**2))),
            "tol_s": tol_s
        })
        sse2d[h] += float(np.sum(d["err2d_m"]**2))
        cnt[h]   += int(len(d))

if not per_run_rows:
    print("No valid pairs even with dynamic tolerance; try raising the upper cap to 120s.")
else:
    per_run = pd.DataFrame(per_run_rows).sort_values(["H_s","RMSE2D_m"])
    display(per_run.head(15))
    macro = per_run.groupby("H_s")["RMSE2D_m"].mean().rename("macro_RMSE2D_m")
    micro = pd.Series({h: (np.sqrt(sse2d[h]/cnt[h]) if cnt[h] else np.nan) for h in HORIZONS},
                      name="micro_RMSE2D_m")
    print("\nMacro 2D RMSE by horizon:"); display(macro)
    print("\nMicro 2D RMSE by horizon:"); display(micro)


Selected runs:


,flight_id,run_id,len,finite_frac,dt_med,duration_s
0,1,0,359,0.997214,10.0,3580.0
732,1584,0,359,0.997214,10.0,3580.0
3630,4981,0,359,0.997214,10.0,3580.0
3634,4985,0,359,0.997214,10.0,3580.0
734,1586,0,359,0.997214,10.0,3580.0
737,1589,0,359,0.997214,10.0,3580.0
3650,5005,0,359,0.997214,10.0,3580.0
3658,5016,0,359,0.997214,10.0,3580.0
3660,5018,0,359,0.997214,10.0,3580.0
2344,3500,0,359,0.997214,10.0,3580.0


,flight_id,run_id,H_s,pairs,RMSE2D_m,RMSE3D_m,tol_s
0,1,0,60,356,2.288736e-10,2.626756e+03,30
18,5005,0,60,356,6.530977e+02,6.578258e+02,30
21,5016,0,60,356,7.068794e+02,7.099563e+02,30
6,4981,0,60,356,1.091845e+03,1.102614e+03,30
3,1584,0,60,356,2.041358e+03,2.117114e+03,30
12,1586,0,60,356,3.355858e+03,3.374505e+03,30
27,3500,0,60,356,1.532069e+04,1.533788e+04,30
15,1589,0,60,356,3.489593e+06,3.489593e+06,30
24,5018,0,60,356,1.448808e+07,1.448808e+07,30
9,4985,0,60,356,3.036531e+13,3.036531e+13,30



Macro 2D RMSE by horizon:


H_s
60     3.036533e+12
120    2.227493e+24
300    8.800841e+59
Name: macro_RMSE2D_m, dtype: float64


Micro 2D RMSE by horizon:


60     9.602355e+12
120    7.043951e+24
300    2.783070e+60
Name: micro_RMSE2D_m, dtype: float64